### 题目1：数据集Retail_Data_Transactions.csv记录了客户ID，交易时间和交易金额三个字段，需求是对客户ID进行汇总统计，计算每一个客户的如下特征变量，并把结果输出为csv文件。  
•	Last_date: 最后一次交易时间   
•	Recency: 最后一次交易时间距离今天的间隔天数  
•	Frequnece: 累计交易次数  
•	Amount: 累计交易金额  
•	Intereval: 平均两次交易之间的间隔天数(如果客户只有一次交易字段为空值)  


In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
from tqdm import tqdm

# 数据预处理

In [2]:
path = r'Retail_Data_Transactions.csv'

In [6]:
# 读取数据
def read_data(path):
    data = pd.read_csv(path)
#     print(data.head)
#     print(data.shape)
    return data

In [4]:
# 替换英文月份
def rep_mon(data):
    data['trans_date'] = data['trans_date'].str.replace('Jan','01')
    data['trans_date'] = data['trans_date'].str.replace('Feb','02')
    data['trans_date'] = data['trans_date'].str.replace('Mar','03')
    data['trans_date'] = data['trans_date'].str.replace('Apr','04')
    data['trans_date'] = data['trans_date'].str.replace('May','05')
    data['trans_date'] = data['trans_date'].str.replace('Jun','06')
    data['trans_date'] = data['trans_date'].str.replace('Jul','07')
    data['trans_date'] = data['trans_date'].str.replace('Aug','08')
    data['trans_date'] = data['trans_date'].str.replace('Sep','09')
    data['trans_date'] = data['trans_date'].str.replace('Oct','10')
    data['trans_date'] = data['trans_date'].str.replace('Nov','11')
    data['trans_date'] = data['trans_date'].str.replace('Dec','12')
    return data

In [5]:
# 转换时间格式
def tran_date(data):
    for i in tqdm(range(len(data))):
        data['trans_date'][i] = datetime.strptime(data['trans_date'][i],"%d-%m-%y")
    return data

In [7]:
if __name__ == '__main__':
    data = read_data(path)
    data = rep_mon(data)
    data = tran_date(data)
    data.head()

  0%|                                                                                       | 0/125000 [00:00<?, ?it/s]<ipython-input-5-18e8878a292f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['trans_date'][i] = datetime.strptime(data['trans_date'][i],"%d-%m-%y")
100%|████████████████████████████████████████████████████████████████████████| 125000/125000 [00:59<00:00, 2093.04it/s]


# 计算特征变量
## Last_date: 最后一次交易时间

In [230]:
# 根据用户id分组获取最后交易时间
def last_date(data):
    result_date = pd.DataFrame(data.groupby('customer_id')['trans_date'].max())
    result_date.columns=['last_date']
    return result_date
result_data = last_date(data)

## Recency: 最后一次交易时间距离今天的间隔天数

In [231]:
# 用今天的日期减去最后交易日期
def recency(result_data):
    today = datetime.today()
    result_data['recency'] = [0]*len(result_data)
    for i in range(len(result_data)):
        result_data['recency'][i] = (today-result_data['last_date'][i]).days
    return result_data
result_data = recency(result_data)

<ipython-input-231-b99cd627f974>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_data['recency'][i] = (today-result_data['last_date'][i]).days


## Frequnece: 累计交易次数

In [232]:
# 根据用户id分组统计总次数，并利用id与result表连接
def frequence(data,result):
    fre = data.groupby('customer_id').tran_amount.count()
    result_data = result.merge(fre,how='left',left_on='customer_id',right_on='customer_id')
    result_data = result_data.rename(columns={'tran_amount':'frequence'})
    return result_data
result_data = frequence(data,result_data)

##  Amount: 累计交易金额

In [233]:
# 根据用户id分组统计累计交易金额，并利用id与result表连接
def amount(data_,result_):
    amo = data.groupby('customer_id').tran_amount.sum()
    result_data = result_.merge(amo,how='left',left_on='customer_id',right_on='customer_id')
    result_data = result_data.rename(columns={'tran_amount':'amount'})
    return result_data
result_data = amount(data,result_data)

## Intereval: 平均两次交易之间的间隔天数(如果客户只有一次交易字段为空值)

In [234]:
# 将原始数据按照用户id分组，将用户id设置为索引
# 计算每个用户两次交易之间的间隔天数，并计算平均值
def inter(data__,result__):
    data_group = data.groupby('customer_id').apply(lambda x:x[:])
    data_group = data_group.drop(axis=1,columns='customer_id')
    data_group = data_group.reset_index('customer_id')
    data_group = data_group.set_index('customer_id')
    
    intereval=dict()
    id_index = data_group.index.unique()
    for i in id_index:
        if len(data_group.loc[i])==1:
            intereval[i]=np.nan
        else:
            intereval[i] = np.mean(data_group.loc[i].sort_values('trans_date')['trans_date'].diff()).days
    intereval = pd.DataFrame(intereval,index=['intereval']).T
    result_data = result__.merge(intereval,how='left',left_on='customer_id',right_on=intereval.index)
    return result_data

In [235]:
result_data = inter(data,result_data)
result_data.head()

,customer_id,last_date,recency,frequence,amount,intereval
0,CS1112,2015-01-14,2599,15,1012,93
1,CS1113,2015-02-09,2573,20,1490,71
2,CS1114,2015-02-12,2570,19,1432,72
3,CS1115,2015-03-05,2549,22,1659,62
4,CS1116,2014-08-25,2741,13,857,96


In [236]:
result_data.to_csv('question1.csv',index=False)